# **(Data Collection)**

## Objectives

* Fetch data from Kaggle and save as raw data, prepare it for further process

## Inputs

* Kaggle JSON file-authentication token

## Outputs

*  Generate Dataset:inputs/datasets/cherry-leaves_dataset

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 



In [1]:
import numpy

import os

---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [2]:
import os
current_dir = os.getcwd()
current_dir

'c:\\python_projects\\project_5\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [3]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'c:\\python_projects\\project_5'

In [5]:
#install Kaggle 
!pip install kaggle

Defaulting to user installation because normal site-packages is not writeable


In [6]:
os.environ['KAGGLE_CONFIG_DIR']=os.getcwd()
! chmod 600 kaggle.json

'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
KaggleDatasetPath = " "
DestinationFolder = "inputs/cherry-leaves_dataset"

os.makedirs(DestinationFolder, exist_ok=True)
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

usage: kaggle datasets download [-h] [-f FILE_NAME] [-p PATH] [-w] [--unzip]
                                [-o] [-q]
                                [dataset]
kaggle datasets download: error: argument -d/--dataset: expected one argument


In [8]:
import os
print("Current Working Directory:", os.getcwd())

Current Working Directory: c:\python_projects\project_5


In [9]:
DestinationFolder = os.path.abspath("inputs/cherry-leaves_dataset")
zip_file_path = os.path.join(DestinationFolder, "cherry-leaves.zip")

In [10]:
import zipfile
DestinationFolder = os.path.abspath("inputs/cherry-leaves_dataset")
zip_file_path = os.path.join(DestinationFolder, "cherry-leaves.zip")

if not os.path.exists(zip_file_path):
    print(f"Error: File not found at {zip_file_path}")
else:
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(DestinationFolder)
        print(f"Extracted: {zip_file_path}")
        
        os.remove(zip_file_path)  # Remove the ZIP file after extraction
        print(f"Deleted: {zip_file_path}")
    except Exception as e:
        print(f"Error extracting ZIP file: {e}")

Error: File not found at c:\python_projects\project_5\inputs\cherry-leaves_dataset\cherry-leaves.zip


Data Cleaning
 Check and remove non-image files

In [11]:
def remove_non_image_file(my_data_dir):
    image_extenstion=('.png','.jpg','.jpeg', '.bmp')
    folders=os.listdir(my_data_dir)
    for folder in folders:
        files=os.listdir(my_data_dir + '/' + folder)
        # print (files)
        i=[]
        j=[]
        for given_file in files:
            if not given_file.lower().endswith(image_extenstion):
                file_location=my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location) #remove non image file
                i.append(1)
            else:
                j.append(1)
        print(f"Folder:{folder} - has image file", len(j))
        print(f"Folder:{folder} - has image file", len(i))
        


In [12]:
remove_non_image_file(my_data_dir='inputs/cherry-leaves_dataset/cherry-leaves')

Folder:healthy - has image file 0
Folder:healthy - has image file 0
Folder:powdery_mildew - has image file 0
Folder:powdery_mildew - has image file 0


PermissionError: [WinError 5] Åtkomst nekad: 'inputs/cherry-leaves_dataset/cherry-leaves/test/healthy'

Split train validation test set 

In [ ]:
!pip install joblib


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\omer\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import shutil
import random

In [13]:
import os
import shutil
import random
import joblib

def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):

    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
        print("train_set_ratio + validation_set_ratio + test_set_ratio should sum to 1.0")
        return

    # gets classes labels
    labels = os.listdir(my_data_dir)  # it should get only the folder name
    if 'test' in labels:
        pass
    else:
        # create train, test folders with classes labels sub-folder
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(name=my_data_dir + '/' + folder + '/' + label)

        for label in labels:

            files = os.listdir(my_data_dir + '/' + label)
            random.shuffle(files)

            train_set_files_qty = int(len(files) * train_set_ratio)
            validation_set_files_qty = int(len(files) * validation_set_ratio)
            count = 1
            for file_name in files:
                if count <= train_set_files_qty:
                    # move a given file to the train set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/train/' + label + '/' + file_name)

                elif count <= (train_set_files_qty + validation_set_files_qty):
                    # move a given file to the validation set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/validation/' + label + '/' + file_name)

                else:
                    # move given file to test set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/test/' + label + '/' + file_name)

                count += 1

            os.rmdir(my_data_dir + '/' + label)

            

In [14]:
split_train_validation_test_images(
    my_data_dir='inputs/cherry-leaves_dataset/cherry-leaves',
    train_set_ratio=0.7,
    validation_set_ratio=0.1,
    test_set_ratio=0.2
)

In [15]:
import os
import shutil
import random

my_data_dir='inputs/cherry-leaves_dataset/cherry-leaves'
classes=['healthy', 'powdery_mildew']
splits=['train', 'validation', 'test']
split_ratios=[0.7,0.1,0.2]

# Make sure destination folders exist
for split in splits:
    for cls in classes:
        split_path = os.path.join(my_data_dir, split, cls)
        os.makedirs(split_path, exist_ok=True)

    # Function to split and move images
def split_and_move_images():
    for cls in classes:
        src_folder = os.path.join(my_data_dir, cls)
        all_images = [f for f in os.listdir(src_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        random.shuffle(all_images)

        total = len(all_images)
        train_end = int(total * split_ratios[0])
        val_end = train_end + int(total * split_ratios[1])

        train_files = all_images[:train_end]
        val_files = all_images[train_end:val_end]
        test_files = all_images[val_end:]

        for split, file_list in zip(splits, [train_files, val_files, test_files]):
            for filename in file_list:
                src_path = os.path.join(src_folder, filename)
                dst_path = os.path.join(my_data_dir, split, cls, filename)
                shutil.move(src_path, dst_path)

        print(f"{cls}: {len(train_files)} train, {len(val_files)} validation, {len(test_files)} test")

split_and_move_images()

healthy: 0 train, 0 validation, 0 test
powdery_mildew: 0 train, 0 validation, 0 test


---

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* If you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.